In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import math
import csv
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('pongngai', 'nkOGg4Ym3J1JFb6yu27O')
%matplotlib inline

Task 0: Loading data to DataFrame

In [3]:
#Read CSV file from the url to DataFrame
url = "https://github.com/Brunel-Visualization/Brunel/raw/master/python/examples/data/whiskey.csv"
s = requests.get(url).text
whisky = pd.read_csv(StringIO(s))

Task 1: Looking around dataset

In [38]:
whisky.shape

(283, 8)

In [5]:
whisky.head()

,Name,Rating,Country,Category,Price,ABV,Age,Brand
0,Canadian Hunter Canadian Whisky,40.0,Canada,Blended,9.0,40.0,NaN,Canadian Hunter
1,Canadian LTD Blended Canadian Whiskey,43.0,Canada,Blended,10.0,NaN,NaN,Canadian LTD
2,Kellan Irish Whiskey,47.0,Ireland,Blended,20.0,40.0,NaN,Kellan
3,Rich & Rare Canadian Whisky,47.0,Canada,Blended,10.0,NaN,NaN,Rich & Rare
4,Canadian Mist Blended Canadian Whisky,48.0,Canada,Blended,12.0,40.0,NaN,Canadian Mist


In [6]:
whisky.describe()

,Rating,Price,ABV,Age
count,272.000000,279.000000,270.000000,174.000000
mean,84.474265,72.483871,44.610444,14.339080
std,11.877887,83.992242,5.883056,6.322267
min,40.000000,2.000000,35.500000,0.000000
25%,80.000000,30.000000,40.000000,10.000000
50%,88.000000,50.000000,43.000000,14.500000
75%,94.000000,80.000000,46.000000,18.000000
max,100.000000,850.000000,68.200000,40.000000


In [11]:
whisky.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 8 columns):
Name        283 non-null object
Rating      272 non-null float64
Country     283 non-null object
Category    283 non-null object
Price       279 non-null float64
ABV         270 non-null float64
Age         174 non-null float64
Brand       283 non-null object
dtypes: float64(4), object(4)
memory usage: 17.8+ KB


In [19]:
categories = set(whisky['Category'])
countries = set(whisky['Country'])
names = set(whisky['Name'])
brands = set(whisky['Brand'])

In [32]:
print("Categories:", categories)

Categories: {'Speyside', 'Single Malt', 'Flavored', 'Blended', 'Unaged', 'Pure Pot Still', 'Corn', 'Campbeltown', 'Rye', 'Grain', 'Lowlands', 'Bourbon', 'Islands', 'Islay', 'Highlands'}


In [33]:
print("Counties:", countries)

Counties: {'Ireland', 'England', 'India', 'Scotland', 'USA', 'Taiwan', 'France', 'Japan', 'Canada'}


In [34]:
print("Brands:", brands)

Brands: {'Smooth Ambler', 'Chivas Regal', 'Old Forester', 'George T. Stagg', 'Lagavulin', 'Famous Grouse', 'William Larue Weller', 'Ellington', 'Old Pulteney', 'Kingdom', 'Midleton', 'Collingwood', 'Cedar Ridge', 'Michael Collins', 'Mickey Finn', "Wiser's", 'P & M', 'Slane Castle', 'Connemara', 'Royal Canadian', 'Aberfeldy', 'Powers', 'Breckenridge', 'William Grant & Sons', 'Buffalo Trace', 'Downslope', 'Talisker', 'Elijah Craig', 'Speyburn', '8 Seconds', 'Glendullan', 'Canadian Hunter', 'Balcones', 'Eagle', "Jack Daniel's", 'Kavalan', 'Blanton', "Bell's", 'Hudson', 'Middle West', 'High West', 'Suntory', 'Ancnoc', 'Pescadores', 'Kellan', 'A.H. Hirsch', 'Macallan', 'Oban', 'Isle of Jura', 'Cardhu', 'Yamazaki', 'Glenmorangie', "Russell's", 'Stillhouse', 'Feckin', 'Pappy Van Winkle', 'Tobermory', 'Redbreast', 'Delaware Phoenix', 'Bunnahabhain', 'Canadian LTD', 'The Irishman', 'Kansas Spirit', 'Thomas H. Handy', 'Benchmark', "Noah's Mill", 'Wolfhound', 'Amrut', 'Knob creek', 'Black Maple H

In [35]:
print("First 10 names:", list(names)[:10])

First 10 names: ['Elmer T Lee Bourbon Whiskey', 'Michael Collins 10 Year Old Single Malt Irish Whiskey', 'Bushmills Original Irish Whiskey', 'Jim Beam Kentucky Straight Bourbon Whiskey', 'Aberlour 18 Year Old Single Malt Scotch', "Booker's True Barrel Bourbon", 'Hudson Single Malt Whiskey', 'Tullamore Dew Irish Whiskey', 'Four Roses Single Barrel Kentucky Straight Bourbon', 'Crown Royal Canadian Whisky']


Task 2: Investigae missing data

In [36]:
data = whisky.values

In [52]:
#columns 1, 4- 6 are numeric
x = np.concatenate((data[:,1].reshape(-1,1), data[:,4:7]),axis=1)
print(x.shape)

(283, 4)


In [53]:
#Find median of each numeric attribute
med_rating = np.median([xx for xx in x[:,0] if not math.isnan(xx)])
med_price = np.median([xx for xx in x[:,1] if not math.isnan(xx)])
med_avb = np.median([xx for xx in x[:,2] if not math.isnan(xx)])
med_age = np.median([xx for xx in x[:,3] if not math.isnan(xx)])

print("Median's rating:", med_rating)
print("Median's price:", med_price)
print("Median's avb:", med_avb)
print("Median's age:", med_age)

Median's rating: 88.0
Median's price: 50.0
Median's avb: 43.0
Median's age: 14.5


In [56]:
#replace nan with median
rating = np.array([med_rating if math.isnan(xx) else xx for xx in x[:,0]]).reshape(-1,1)
price = np.array([med_price if math.isnan(xx) else xx for xx in x[:,1]]).reshape(-1,1)
avb = np.array([med_avb if math.isnan(xx) else xx for xx in x[:,2]]).reshape(-1,1)
age = np.array([med_age if math.isnan(xx) else xx for xx in x[:,3]]).reshape(-1,1)
x = np.concatenate((rating, price, avb, age), axis=1)
print("Median after replacing:",np.median(x,axis=0))

Median after replacing: [88.  50.  43.  14.5]


Task 3: Investigae missing data

In [57]:
def outlier(x):
    rating_q1 = np.percentile(x,25)
    rating_q3 = np.percentile(x,75)
    IQR = rating_q3 - rating_q1
    lower_outlier = rating_q1 - (1.5*IQR)
    higher_outlier = rating_q3 + (1.5*IQR)
    return (lower_outlier, higher_outlier)

In [87]:
#Define the function that return index of outliers
def del_outlier(x):
    l, h = outlier(x)
    return [i for i,xx in enumerate(x) if xx < l or xx > h], l, h

In [88]:
#Create index of outlier in each attribute
rating_idx, lrate, hrate = del_outlier(x[:,0])
price_idx, lprice, hprice = del_outlier(x[:,1])
avb_idx, lavb, havb = del_outlier(x[:,2])
age_idx, lage, hage = del_outlier(x[:,3])

In [89]:
print("Rating => lower outlier:", lrate, ",upper outlier:", hrate)
print("Price => lower outlier:", lprice, ",upper outlier:", hprice)
print("AVB => lower outlier:", lavb, ",upper outlier:", havb)
print("Age => lower outlier:", lage, ",upper outlier:", hage)

Rating => lower outlier: 63.0 ,upper outlier: 111.0
Price => lower outlier: -44.25 ,upper outlier: 153.75
AVB => lower outlier: 31.0 ,upper outlier: 55.0
Age => lower outlier: 7.5 ,upper outlier: 19.5


In [117]:
print("Rating outlier index:", rating_idx,"\n")
print("Price outlier index:", price_idx,"\n")
print("AVB outlier index:", avb_idx,"\n")
print("Age outlier index:", age_idx,"\n")
outlier_idx = set(np.concatenate((rating_idx, price_idx, avb_idx, age_idx)))
print("All outlier index: ", outlier_idx)

Rating outlier index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] 

Price outlier index: [54, 98, 106, 109, 143, 152, 167, 168, 170, 171, 172, 186, 209, 214, 220, 225, 226, 232, 241, 271, 275, 280] 

AVB outlier index: [59, 67, 68, 75, 88, 136, 137, 143, 144, 154, 174, 188, 207, 222, 225, 239, 244, 266, 268, 269, 276, 282] 

Age outlier index: [6, 8, 12, 21, 30, 31, 37, 45, 46, 67, 80, 81, 87, 94, 109, 114, 129, 135, 142, 150, 152, 155, 166, 168, 170, 172, 174, 184, 186, 187, 199, 205, 206, 208, 209, 211, 214, 217, 220, 225, 226, 228, 232, 241, 249, 266, 274, 275] 

All outlier index:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 21, 30, 31, 37, 45, 46, 54, 59, 67, 68, 75, 80, 81, 87, 88, 94, 98, 106, 109, 114, 129, 135, 136, 137, 142, 143, 144, 150, 152, 154, 155, 166, 167, 168, 170, 171, 172, 174, 184, 186, 187, 188, 199, 205, 206, 207, 208, 209, 211, 214, 217, 220, 222, 225, 226, 228, 232, 239, 241, 244, 249, 266, 268, 269, 271, 274, 275, 276, 280, 282}


Task 4: Plot

In [106]:
#Zzzzz

Task 5: Discover pattern

- Bourbon age >= 4
- Rye age >= 2

In [114]:
#bourbon age must >= 4 years
bour_idx = [i for i, x in whisky.iterrows() if x['Category'] == 'Bourbon' and x['Age'] < 4]

#rye age must >= 2 years
rye_idx = [i for i, x in whisky.iterrows() if x['Category'] == 'Rye' and x['Age'] < 2]

In [115]:
print("less than 4 yrs Bourbon index:", bour_idx)
print("less than 2 yrs Rye index:", rye_idx)

less than 4 yrs Bourbon index: [155]
less than 2 yrs Rye index: [45, 228]
